In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import tensorflow as tf
import keras.utils
import numpy as np
import cv2
import pandas as pd

#Load Data


In [3]:
#df = pd.read_csv('/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/ImageCLEFmedical_Caption_2023_concept_detection_train_labels.csv', sep="\t")

# Karthik
df = pd.read_csv('/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/ImageCLEFmedical_Caption_2023_concept_detection_train_labels.csv', sep="\t")

In [4]:
df

,ID,cuis
0,ImageCLEFmedical_Caption_2023_train_000001,C0040405
1,ImageCLEFmedical_Caption_2023_train_000002,C0041618
2,ImageCLEFmedical_Caption_2023_train_000003,C0040405;C0817096;C0205271
3,ImageCLEFmedical_Caption_2023_train_000004,C0002978;C0036033;C0262950;C0225317
4,ImageCLEFmedical_Caption_2023_train_000005,C0040405;C0817096;C0497156
...,...,...
60913,ImageCLEFmedical_Caption_2023_train_060914,C0040405;C0005682
60914,ImageCLEFmedical_Caption_2023_train_060915,C1306645;C0000726;C1999039;C0203102;C0022646;C...
60915,ImageCLEFmedical_Caption_2023_train_060916,C0002978;C1510412;C0226333
60916,ImageCLEFmedical_Caption_2023_train_060917,C0002978;C1510412


#Find number of Classes

In [5]:
all_cui_train=df.loc[:,"cuis"]

In [6]:

sep_cui_train=[]
for cui_row in all_cui_train:
  #print(type(cui_row))
  sep_cui_train.extend(cui_row.split(sep=";"))
  
  
print(len(sep_cui_train))
unique_cui_train=np.array(sep_cui_train)
print(len(np.unique(unique_cui_train)))

227156
2125


In [8]:
# df_valid = pd.read_csv('/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/ImageCLEFmedical_Caption_2023_concept_detection_valid_labels.csv', sep="\t")

# Karthik
df_valid = pd.read_csv('/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/ImageCLEFmedical_Caption_2023_concept_detection_valid_labels.csv', sep="\t")

In [ ]:
df_valid.head()

,ID,cuis
0,ImageCLEFmedical_Caption_2023_valid_000001,C1306645;C0817096;C1996865;C0442800;C0018787;C...
1,ImageCLEFmedical_Caption_2023_valid_000002,C1306645;C0030797;C0016169;C0227668;C0087086
2,ImageCLEFmedical_Caption_2023_valid_000003,C1306645;C0006141;C0027651;C0332482;C0206692;C...
3,ImageCLEFmedical_Caption_2023_valid_000004,C0040405;C1306460
4,ImageCLEFmedical_Caption_2023_valid_000005,C0040405;C0035359;C0027651


In [9]:
df.loc[1875]

ID      ImageCLEFmedical_Caption_2023_train_001876
cuis           C1306645;C0817096;C0205129;C0039985
Name: 1875, dtype: object

In [10]:
all_cui_valid=df_valid.loc[:,"cuis"]

In [11]:
sep_cui_valid=[]
for cui_row in all_cui_valid:
  #print(type(cui_row))
  sep_cui_valid.extend(cui_row.split(sep=";"))
  
  
print(len(sep_cui_valid))
unique_cui_valid=np.array(sep_cui_valid)
print(len(np.unique(unique_cui_valid)))

40042
1945


In [12]:
train_and_valid_cuis=[]
train_and_valid_cuis.extend(unique_cui_train)
train_and_valid_cuis.extend(unique_cui_valid)

In [13]:
total_cui=np.array(train_and_valid_cuis)
print(len(np.unique(total_cui)))

2125


2125 is the total number of unique classes

#Label Encoding

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
encoder=LabelEncoder()
encoder.fit(total_cui)

LabelEncoder()

In [16]:
print(len(encoder.classes_))

2125


In [17]:
print(encoder.classes_)

['C0000726' 'C0000833' 'C0000846' ... 'C5396784' 'C5441965' 'C5574908']


In [19]:
# base_path='/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023'

# Karthik
base_path = '/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023'

In [20]:
file_path = os.path.join(base_path,'cui_classes.npy')
np.save(file_path, encoder.classes_)
print("Saved", file_path)

Saved /content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/cui_classes.npy


In [21]:
data = np.load(os.path.join(base_path, 'cui_classes.npy'))

In [22]:
print(data[0])

C0000726


In [23]:
print(type(data))

<class 'numpy.ndarray'>


#Input Sequencer

In [24]:
df_train_copy=df

In [25]:
img_cui_dict=df_train_copy.set_index('ID').T.to_dict('list')

In [ ]:
#print(img_cui_dict['ImageCLEFmedical_Caption_2023_train_001866'])

In [35]:
# image_path='/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train'

# Karthik
image_path = '/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train'

In [36]:
!ls /content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train | grep ".*\.jpg" | wc -l

ls: cannot access '/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train': No such file or directory
0


#saving ids

In [37]:
#no need to run already saved
# img_id=[]
# for image in os.listdir(image_path):
#   img_id.append(image)

In [38]:
#no need to run already saved
# file_path_img_id = os.path.join(base_path,'img_id.npy')
# np.save(file_path_img_id, np.array(img_id))
# print("Saved", file_path_img_id)

Saved /content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/img_id.npy


#Load Ids from file

In [39]:
img_id = np.load(os.path.join(base_path, 'img_id.npy'))

In [47]:
print(len(img_id))
print(img_id[10])

61019
ImageCLEFmedical_Caption_2023_train_039154.jpg


In [48]:
print(img_cui_dict['ImageCLEFmedical_Caption_2023_train_039154.jpg'[:-4]][0].split(sep=";"))

['C1306645', 'C0817096', 'C1996865']


In [87]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
enc_labels=[]
for i in range(0,2125):
  enc_labels.append(i)
print(enc_labels)
mlb.fit([enc_labels])
print("Transformed:", mlb.transform([(661,2124,1),(11, 22)]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [84]:
enc_tu=tuple([661])
print(enc_tu)
dd=[]
dd.append(enc_tu)
print(dd)

(661,)
[(661,)]


In [119]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, encoder=None, mlb=None, batch_size=8, image_size=224):
        self.ids = ids
        self.batch_size = batch_size
        self.image_size = image_size
        self.trainpath = '/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train'
        
        if encoder is not None:
          self.encoder = encoder
        if mlb is not None:
          self.mlb = mlb

        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path ='/content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train'+'/'+id_name             
        
        ## Reading Image        
        image = cv2.imread(image_path, 1)
        cui_ids = img_cui_dict[id_name[:-4]][0].split(sep=";")        
        
        if(image is None):
          print("Error: ",id_name)
        image = cv2.resize(image, (self.image_size, self.image_size)).astype('float32')

        # Convert to float
        image /= 255
        return image, cui_ids
    
    def __getitem__(self, index):#For returning batch of images and thier cui ids
        # if(index+1)*self.batch_size > len(self.ids):
        #     self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        #print(len(files_batch))
        if(len(files_batch)<self.batch_size):
          files_batch.extend(self.ids[0:self.batch_size-len(files_batch)])
        
        images = []
        batch_cuis  = []
        for i,id_name in enumerate(files_batch):

          # while(not id_name.split('.')[1]=='jpg'): #to check if its not a jpg image
          #   id_name=files_batch[i+1]
          #   i+=1
          #   if(i>=self.batch_size):
          #     i=0
          
          _img, _cui_temp = self.__load__(id_name)
          images.append(_img)          
          batch_cuis.append(self.encoder.transform(_cui_temp))           
            
        images = np.array(images)
        encoded_batch_cuis = np.array(self.mlb.transform(batch_cuis))

        # print("Batch Encoded CUI IDs", encoded_batch_cuis)
        return images, encoded_batch_cuis
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [111]:
test_gen = DataGen(img_id, encoder, mlb)

In [112]:
test_gen.__getitem__(0)

(8, 2125)


(array([[[[0.3372549 , 0.3372549 , 0.3372549 ],
          [0.3137255 , 0.3137255 , 0.3137255 ],
          [0.3137255 , 0.3137255 , 0.3137255 ],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         [[0.28235295, 0.28235295, 0.28235295],
          [0.2627451 , 0.2627451 , 0.2627451 ],
          [0.28235295, 0.28235295, 0.28235295],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         [[0.31764707, 0.31764707, 0.31764707],
          [0.28627452, 0.28627452, 0.28627452],
          [0.28235295, 0.28235295, 0.28235295],
          ...,
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
 
         ...,
 
         [[0.        , 0.        , 0.        ],
          [0.     

#Verifying Label Encoding

In [120]:
print(img_cui_dict['ImageCLEFmedical_Caption_2023_train_052471.jpg'[:-4]][0].split(sep=";"))

['C0040405']


In [121]:
encoder.transform(['C0040405'])

array([661])

In [122]:
mlb.transform([(661,)])[0][661]

1

Start with effecient net B0, in case of multiple GPU's Resnet 101 DenseNet 121

# Model Training

In [123]:
data_reader = DataGen(img_id, encoder, mlb, batch_size=8)
img,labels = data_reader[5]
print(img.shape)
print(labels)

(8, 224, 224, 3)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [124]:
print(keras.__version__)
print(tf.__version__)

2.12.0
2.12.0


In [78]:
!pip install EfficientNet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from keras import backend as K
#from efficientnet.efficientnet.model import EfficientNetB0
import efficientnet.keras

from efficientnet.keras import EfficientNetB0
IMG_SIZE=(224,224)

#model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
model = EfficientNetB0(
        include_top=False,
        #weights='imagenet',
        input_shape=(*IMG_SIZE, 3)
       )
#model.summary(line_length=150)

flatten = Flatten()
new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')

inp2 = model.input
out2 = new_layer2(flatten(model.output))

opt = keras.optimizers.Adam(learning_rate=1e-05)


model2 = Model(inp2, out2)
model2.summary()
model2.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc']
)


#weight_save = keras.callbacks.ModelCheckpoint('weights/weights-efficientnetb0/weights-epoch-{epoch:03d}.h5', save_weights_only=True, period=1)
#on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())

#model2.load_weights('/home/miruna/LifeCLEF/FungiCLEF/weights/weights-efficientnetb0/weights-epoch-006.h5')
model2.fit(data_reader,
    epochs=10,
    verbose=1,
    steps_per_epoch=31000,
    #callbacks=[weight_save, on_epoch_end_call]
)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 112, 112, 32  128         ['stem_conv[0][0]']              
                                )                                                           

In [ ]:
tf.keras.applications.densenet.DenseNet121(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

33188688/33188688 [==============================] - 2s 0us/step


In [80]:
from sklearn.preprocessing import MultiLabelBinarizer

In [81]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from keras import backend as K

IMG_SIZE=(224,224)

In [82]:
# model.summary(line_length=150)
from tensorflow.keras.applications.densenet import DenseNet121




model = DenseNet121(
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
       )
flatten = Flatten()
new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')

inp2 = model.input
out2 = new_layer2(flatten(model.output))

opt = keras.optimizers.Adam(learning_rate=1e-05)
data_reader = DataGen(img_id)
model2 = Model(inp2, out2)
model2.summary()
model2.compile(
    optimizer=opt,
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)


#weight_save = keras.callbacks.ModelCheckpoint('weights/weights-efficientnetb0/weights-epoch-{epoch:03d}.h5', save_weights_only=True, period=1)
#on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())

#model2.load_weights('/home/miruna/LifeCLEF/FungiCLEF/weights/weights-efficientnetb0/weights-epoch-006.h5')
model2.fit(data_reader,
    epochs=10,
    verbose=1,
    steps_per_epoch=31000,
    #callbacks=[weight_save, on_epoch_end_call]
)

29084464/29084464 [==============================] - 0s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_3[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                           

ValueError: ignored